<a href="https://colab.research.google.com/github/A-b-d-u-l-H-a-n-a-n/Large_Language_Models/blob/main/Large_Language_Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets evaluate transformers[sentencepiece]
!pip install accelerate
!apt install git-lfs

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.1 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which i

In [1]:
!git config --global user.email "hanan6184@gmail.com"
!git config --global user.name "A-b-d-u-l-H-a-n-a-n"

In [3]:
from huggingface_hub import notebook_login

notebook_login()

In [4]:
from transformers import AutoModelForMaskedLM

model_checkpoint = "distilbert-base-uncased"
model = AutoModelForMaskedLM.from_pretrained(model_checkpoint)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [5]:
distilbert_num_parameters = model.num_parameters() / 1_000_000
print(f"'>>> DistilBERT number of parameters: {round(distilbert_num_parameters)}M'")
print(f"'>>> BERT number of parameters: 110M'")

'>>> DistilBERT number of parameters: 67M'
'>>> BERT number of parameters: 110M'


In [6]:
text = "This is a great [MASK]."

In [7]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, max_length=512)


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [8]:
import torch

inputs = tokenizer.encode_plus(
    text,
    add_special_tokens=True,
    max_length=512,
    truncation=True,
    padding="max_length",
    return_attention_mask=True,
    return_tensors="pt"
)
token_logits = model(**inputs).logits
# Find the location of [MASK] and extract its logits
mask_token_index = torch.where(inputs["input_ids"] == tokenizer.mask_token_id)[1]
mask_token_logits = token_logits[0, mask_token_index, :]
# Pick the [MASK] candidates with the highest logits
top_5_tokens = torch.topk(mask_token_logits, 5, dim=1).indices[0].tolist()

for token in top_5_tokens:
    print(f"'>>> {text.replace(tokenizer.mask_token, tokenizer.decode([token]))}'")

'>>> This is a great deal.'
'>>> This is a great success.'
'>>> This is a great adventure.'
'>>> This is a great idea.'
'>>> This is a great feat.'


In [9]:
import torch
from torch.utils.data import DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup, default_data_collator
from datasets import load_dataset

In [10]:
dataset = load_dataset("spoiled/with_random_label")

Generating train split:   0%|          | 0/53036 [00:00<?, ? examples/s]

In [11]:
print(dataset['train'].features)
dataset

{'text': Value(dtype='string', id=None), 'label': Value(dtype='int64', id=None)}


DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 53036
    })
})

In [12]:
sample = dataset["train"].shuffle(seed=42).select(range(3))

for row in sample:
    print(f"\n'>>> Review: {row['text']}'")
    print(f"'>>> Label: {row['label']}'")


'>>> Review: Very big is not a personality trait.'
'>>> Label: 0'

'>>> Review: Increased knowledge is to enhance knowledge or improve knowledge.'
'>>> Label: 0'

'>>> Review: Sweating is not related to losing weight.'
'>>> Label: 0'


In [13]:
def tokenize_function(examples):
    result = tokenizer(examples["text"])
    if tokenizer.is_fast:
        result["word_ids"] = [result.word_ids(i) for i in range(len(result["input_ids"]))]
    return result


# Use batched=True to activate fast multithreading!
tokenized_datasets = dataset.map(
    tokenize_function, batched=True, remove_columns=["text", "label"]
)
tokenized_datasets

Map:   0%|          | 0/53036 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids'],
        num_rows: 53036
    })
})

In [14]:
chunk_size = 128

In [15]:
# Slicing produces a list of lists for each feature
tokenized_samples = tokenized_datasets["train"][:3]

for idx, sample in enumerate(tokenized_samples["input_ids"]):
    print(f"'>>> Review {idx} length: {len(sample)}'")

'>>> Review 0 length: 13'
'>>> Review 1 length: 24'
'>>> Review 2 length: 24'


In [16]:
concatenated_examples = {
    k: sum(tokenized_samples[k], []) for k in tokenized_samples.keys()
}
total_length = len(concatenated_examples["input_ids"])
print(f"'>>> Concatenated reviews length: {total_length}'")

'>>> Concatenated reviews length: 61'


In [17]:
chunks = {
    k: [t[i : i + chunk_size] for i in range(0, total_length, chunk_size)]
    for k, t in concatenated_examples.items()
}

for chunk in chunks["input_ids"]:
    print(f"'>>> Chunk length: {len(chunk)}'")

'>>> Chunk length: 61'


In [18]:
def group_texts(examples):
    # Concatenate all texts
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    # Compute length of concatenated texts
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the last chunk if it's smaller than chunk_size
    total_length = (total_length // chunk_size) * chunk_size
    # Split by chunks of max_len
    result = {
        k: [t[i : i + chunk_size] for i in range(0, total_length, chunk_size)]
        for k, t in concatenated_examples.items()
    }
    # Create a new labels column
    result["labels"] = result["input_ids"].copy()
    return result

In [19]:
lm_datasets = tokenized_datasets.map(group_texts, batched=True)
lm_datasets

Map:   0%|          | 0/53036 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 5277
    })
})

In [20]:
tokenizer.decode(lm_datasets["train"][1]["input_ids"])

'[CLS] a business restaurant is likely to be located in a mall. [SEP] [CLS] business sector isnt a place where a business restaurant can be located. [SEP] [CLS] yellow pages is a listing of businesses. [SEP] [CLS] mouth is an organ in a body used to eat food. [SEP] [CLS] mouth is an organ in a body used to eat food. [SEP] [CLS] grocery cart is used to carry grocery items. [SEP] [CLS] grocery cart is used to carry grocery items. [SEP] [CLS] fruit basket is a container to carry fruit. [SEP] [CLS] fruit basket is a container to carry fruit. [SEP] [CLS] fruit market is a market where various kinds of fruits are sold. [SEP] [CLS]'

In [21]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

In [22]:
samples = [lm_datasets["train"][i] for i in range(2)]
for sample in samples:
    _ = sample.pop("word_ids")

for chunk in data_collator(samples)["input_ids"]:
    print(f"\n'>>> {tokenizer.decode(chunk)}'")


'>>> [CLS] carpet can [MASK] found in [MASK] back of a person. [SEP] [CLS] refrigerator [MASK] an [MASK]liance or compartment which is artificially [MASK] cool and used to store food and drink [MASK] [SEP] [CLS] refrigerator is an applian [MASK] or compartment which is [MASK]ly [MASK] cool and used [MASK] store food and [MASK] [MASK] [SEP] [CLS] bread [MASK] is a box shaped container for storing [MASK]. [SEP] [CLS] fridge is an applian [MASK] used to store food. [SEP] [CLS] baseball is a sport and not a place to [MASK] things. [SEP] [CLS] a business restaurant is [MASK] to [MASK] located in a town. [SEP] [CLS] a business restaurant is likely [MASK] be located at hotel. [SEP]'

'>>> [CLS] a [MASK] restaurant is likely to be located in a mall. [SEP] [CLS] business sector isnt a place where a [MASK] restaurant can be located. [SEP] [CLS] yellow pages is a listing of businesses. [SEP] [CLS] [MASK] [MASK] an organ in a body used to eat food. [SEP] [CLS] mouth is an organ in a body [MASK] t

In [23]:
import collections
import numpy as np

from transformers import default_data_collator

wwm_probability = 0.2


def whole_word_masking_data_collator(features):
    for feature in features:
        word_ids = feature.pop("word_ids")

        # Create a map between words and corresponding token indices
        mapping = collections.defaultdict(list)
        current_word_index = -1
        current_word = None
        for idx, word_id in enumerate(word_ids):
            if word_id is not None:
                if word_id != current_word:
                    current_word = word_id
                    current_word_index += 1
                mapping[current_word_index].append(idx)

        # Randomly mask words
        mask = np.random.binomial(1, wwm_probability, (len(mapping),))
        input_ids = feature["input_ids"]
        labels = feature["labels"]
        new_labels = [-100] * len(labels)
        for word_id in np.where(mask)[0]:
            word_id = word_id.item()
            for idx in mapping[word_id]:
                new_labels[idx] = labels[idx]
                input_ids[idx] = tokenizer.mask_token_id
        feature["labels"] = new_labels

    return default_data_collator(features)

In [24]:
samples = [lm_datasets["train"][i] for i in range(2)]
batch = whole_word_masking_data_collator(samples)

for chunk in batch["input_ids"]:
    print(f"\n'>>> {tokenizer.decode(chunk)}'")


'>>> [CLS] carpet can be found in the [MASK] of a person. [SEP] [CLS] [MASK] is an appliance or compartment which [MASK] artificially kept cool and used to store food [MASK] drink. [SEP] [CLS] [MASK] [MASK] an appliance or compartment which [MASK] [MASK] [MASK] kept cool and used to store food [MASK] [MASK]. [SEP] [CLS] breadbox is a box shaped container for storing [MASK]. [SEP] [CLS] [MASK] is an appliance used to [MASK] food. [SEP] [CLS] baseball is a sport and not a place [MASK] keep things. [SEP] [CLS] [MASK] business restaurant is [MASK] [MASK] be located in a town. [SEP] [CLS] a [MASK] restaurant is [MASK] to be located at hotel. [SEP]'

'>>> [CLS] a business restaurant is [MASK] to be located in a mall. [SEP] [CLS] business sector isnt a place where a [MASK] restaurant can be [MASK] [MASK] [SEP] [CLS] yellow pages is [MASK] [MASK] [MASK] businesses. [SEP] [CLS] mouth is an organ [MASK] a [MASK] used to eat food. [SEP] [CLS] [MASK] is an organ in a body used to eat food [MASK] 

In [26]:
train_size = 4_000
test_size = int(0.1 * train_size)

downsampled_dataset = lm_datasets["train"].train_test_split(
    train_size=train_size, test_size=test_size, seed=42
)
downsampled_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 4000
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 400
    })
})

In [28]:
from transformers import TrainingArguments

batch_size = 64
# Show the training loss with every epoch
logging_steps = len(downsampled_dataset["train"]) // batch_size
model_name = model_checkpoint.split("/")[-1]

training_args = TrainingArguments(
    output_dir=f"{model_name}-with_random_label",
    overwrite_output_dir=True,
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    push_to_hub=True,
    fp16=False,
    logging_steps=logging_steps,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [29]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=downsampled_dataset["train"],
    eval_dataset=downsampled_dataset["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [30]:
import math

eval_results = trainer.evaluate()
print(f">>> Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

>>> Perplexity: 20.75


In [31]:
trainer.push_to_hub()

events.out.tfevents.1723662731.79a784ad69e8.896.0:   0%|          | 0.00/297 [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.18k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/abdulhanan6184/distilbert-base-uncased-with_random_label/commit/af49e18b7d948562df7348290cfd4952e56f93ca', commit_message='End of training', commit_description='', oid='af49e18b7d948562df7348290cfd4952e56f93ca', pr_url=None, pr_revision=None, pr_num=None)

In [32]:
def insert_random_mask(batch):
    features = [dict(zip(batch, t)) for t in zip(*batch.values())]
    masked_inputs = data_collator(features)
    # Create a new "masked" column for each column in the dataset
    return {"masked_" + k: v.numpy() for k, v in masked_inputs.items()}

In [33]:
downsampled_dataset = downsampled_dataset.remove_columns(["word_ids"])
eval_dataset = downsampled_dataset["test"].map(
    insert_random_mask,
    batched=True,
    remove_columns=downsampled_dataset["test"].column_names,
)
eval_dataset = eval_dataset.rename_columns(
    {
        "masked_input_ids": "input_ids",
        "masked_attention_mask": "attention_mask",
        "masked_labels": "labels",
    }
)

Map:   0%|          | 0/400 [00:00<?, ? examples/s]

In [34]:
from torch.utils.data import DataLoader
from transformers import default_data_collator

batch_size = 64
train_dataloader = DataLoader(
    downsampled_dataset["train"],
    shuffle=True,
    batch_size=batch_size,
    collate_fn=data_collator,
)
eval_dataloader = DataLoader(
    eval_dataset, batch_size=batch_size, collate_fn=default_data_collator
)

In [35]:
from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=5e-5)

In [36]:
from accelerate import Accelerator

accelerator = Accelerator()
model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(
    model, optimizer, train_dataloader, eval_dataloader
)

In [37]:
from transformers import get_scheduler

num_train_epochs = 3
num_update_steps_per_epoch = len(train_dataloader)
num_training_steps = num_train_epochs * num_update_steps_per_epoch

lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

In [38]:
from huggingface_hub import get_full_repo_name

model_name = "distilbert-base-uncased-with_random_label"
repo_name = get_full_repo_name(model_name)
repo_name

'abdulhanan6184/distilbert-base-uncased-with_random_label'

In [39]:
from huggingface_hub import HfApi, snapshot_download

api = HfApi()

# Define the model name and repository name
model_name = "distilbert-base-uncased-with_random_label"
repo_name = "abdulhanan6184/distilbert-base-uncased-with_random_label"

# Download the repository contents
repo_path = snapshot_download(repo_id=repo_name, revision="main", cache_dir=model_name)


Fetching 10 files:   0%|          | 0/10 [00:00<?, ?it/s]

README.md:   0%|          | 0.00/1.28k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/552 [00:00<?, ?B/s]

.gitattributes:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.22k [00:00<?, ?B/s]

(…)t.tfevents.1723662731.79a784ad69e8.896.0:   0%|          | 0.00/297 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.18k [00:00<?, ?B/s]

In [40]:
# Create a repository object to manage the repo
try:
    repo_url = api.create_repo(repo_name, exist_ok=True)
except Exception as e:
    print(f"Repository creation failed: {e}")

In [41]:
from tqdm.auto import tqdm
import torch
import math

progress_bar = tqdm(range(num_training_steps))

for epoch in range(num_train_epochs):
    # Training
    model.train()
    for batch in train_dataloader:
        outputs = model(**batch)
        loss = outputs.loss
        accelerator.backward(loss)

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

    # Evaluation
    model.eval()
    losses = []
    for step, batch in enumerate(eval_dataloader):
        with torch.no_grad():
            outputs = model(**batch)

        loss = outputs.loss
        losses.append(accelerator.gather(loss.repeat(batch_size)))

    losses = torch.cat(losses)
    losses = losses[: len(eval_dataset)]
    try:
        perplexity = math.exp(torch.mean(losses))
    except OverflowError:
        perplexity = float("inf")

    print(f">>> Epoch {epoch}: Perplexity: {perplexity}")

    # Save and upload
    accelerator.wait_for_everyone()
    unwrapped_model = accelerator.unwrap_model(model)
    unwrapped_model.save_pretrained(repo_path, save_function=accelerator.save)
    if accelerator.is_main_process:
        tokenizer.save_pretrained(repo_path)

        # Push to the Hub using HfApi
        api.upload_folder(
            repo_id=repo_name,
            folder_path=repo_path,
            commit_message=f"Training in progress epoch {epoch}"
        )

  0%|          | 0/189 [00:00<?, ?it/s]

>>> Epoch 0: Perplexity: 3.1347835367028023


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

>>> Epoch 1: Perplexity: 2.8845092517303836


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

>>> Epoch 2: Perplexity: 2.806069230439348


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [42]:
from transformers import pipeline

mask_filler = pipeline(
    "fill-mask", model="abdulhanan6184/distilbert-base-uncased-with_random_label"
)

config.json:   0%|          | 0.00/552 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.22k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [44]:
test = "Many people eat [MASK] for breakfast."
preds = mask_filler(test)

for pred in preds:
    print(f">>> {pred['sequence']}")

>>> many people eat meat for breakfast.
>>> many people eat cake for breakfast.
>>> many people eat food for breakfast.
>>> many people eat breakfast for breakfast.
>>> many people eat bread for breakfast.
